# Installs

In [1]:
# !pip install -q tqdm
# !pip install -q seaborn
# !pip install -q datasets
# !pip install -q scikit-learn
# !pip install -q pytorch_lightning
# !pip install -q git+https://github.com/MagedSaeed/tkseem

# Prepare

In [2]:
import re
import os
import shutil
import string
from pathlib import Path

from pyarabic import araby

from sklearn.model_selection import train_test_split

import numpy as np

import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

from tqdm.auto import tqdm

import torchmetrics
from torchmetrics.functional import word_error_rate, char_error_rate

from pytorch_lightning import seed_everything
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import (
    EarlyStopping,
    LearningRateMonitor,
    ModelCheckpoint,
    RichProgressBar,
)

from dotless_arabic import constants
from dotless_arabic.processing import undot, process
from dotless_arabic.tokenizers import CharacterTokenizer
from dotless_arabic.experiments.dots_retrieval.src.models import LitBiLSTMModel
from dotless_arabic.datasets.wikipedia.collect import collect_dataset_for_dots_retreival
# from dotless_arabic.datasets.aggregated.collect import collect_dataset_for_dots_retreival
from dotless_arabic.constants import LETTERS_MAPPING

import datasets
import seaborn as sns
import matplotlib.pyplot as plt
from sacremoses import MosesPunctNormalizer

[2024-04-17 12:12:23,289 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [3]:
seed = 42

In [4]:
# random.seed(seed)     # python random generator
# np.random.seed(seed)  # numpy random generator

# torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)

# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

seed_everything(seed)

Global seed set to 42


42

In [5]:
tqdm.pandas()

# Load and explore the dataset

In [6]:
dataset = list(set(collect_dataset_for_dots_retreival()))
len(dataset)

  0%|          | 0/4636663 [00:00<?, ?it/s]

  0%|          | 0/4636645 [00:00<?, ?it/s]

  0%|          | 0/10867699 [00:00<?, ?it/s]

1440782

In [7]:
train_dataset,test_dataset = train_test_split(dataset,test_size=0.05,shuffle=True,random_state=seed)
len(train_dataset),len(test_dataset)

(1368742, 72040)

In [8]:
train_dataset[:2]

['كانت بداياته مع نادي الطائي و في عام 2015 وقع مع الفيصلي و عاد إلى نادي الطائي 2017 و وقع مع النادي العربي في عام 2018 و انتقل إلى نادي الشعلة في عام 2019 و وقع بعدها مع نادي اللواء',
 'تم تشخيص تقرير تسلسل وراثي من كابيتال بيو ميدلاب من بكين عن العامل المسبب لمريض يبلغ من العمر 41 عامًا، وتم جمع العينة من قبل مستشفى ووهان المركزي إذ تم تشخيصه عن طريق الخطأ بأنه فيروس متلازمة تنفسية حادة شديدة (فيروس السارس التاجي)']

In [9]:
test_dataset[:2]

['بطولة كأس السوبر الألباني 2002 هي النسخة التاسعة من بطولة كأس السوبر الألباني ، لعب يوم 14 سبتمبر 2002 في الاستاد الوطني بتيرانا ، بين فريق تيرانا الفائز بكأس ألبانيا وفريق دينامو تيرانا الفائز بالدوري',
 'وهذا المرسى من أحسن المراسي وضعًا، وهو شبه خليج من البحر يدخل في البر، والبر مطيف بحافتيه، ويُكّن من جميع الأرواح، وباستقرارنا فيه عادت لأجسادنا الأرواح، وأمنِّا في مركبنا من اختلال الدسر والألواح']

In [10]:
train_dataset,val_dataset = train_test_split(train_dataset,test_size=0.05,shuffle=True,random_state=seed)
len(train_dataset),len(val_dataset)

(1300304, 68438)

In [11]:
val_dataset[:2]

['لم يبق آل حرز بعد وفاة والده طويلاً إذ توفي شاباً سنة 1340 هـ / 1921 م، ودفن بجوار والده في مقبرة الإمام بجد حفص، ولم يعقب إلا بنتاً واحدة تزوجها محمد علي المدني',
 'كانت بداية مسلسل أخي العزيز عام 1975، حيث كانت على شكل قصة مصورة كما يسميه اليابانيون مانغا، إلى أن تحول إلى مسلسل تلفزيوني عام 1990 وعرض لأول مرة على قناة NHK اليابانية من 1991 - 1992، ودبلج هذا المسلسل إلى اللغة الإيطالية والفرنسية والألمانية والعربية، إلا أنه لاقى سخطا كبيرا حيث أنه منع في بعض الدول كفرنسا لما قيل أنه سبب دمار لعقلية الأطفال، بسبب بعض المشاهد التي تتنافى مع الأخلاق، إلا أنه حافظ على شعبيته في بعض الدول كأمريكا والبرازيل على الرغم من أنه لم يدبلج هناك بشكل كامل وإنما كتابة أسفل الشاشة، مع موسيقى رائعة جدا']

dataset chars and their counts

In [12]:
chars_dict = {}
for document in tqdm(dataset):
  for word in document.split():
      for c in word:
        chars_dict[c] = chars_dict.get(c,0)+1
f'{len(chars_dict.keys()):,}',f'{sum(chars_dict.values()):,}'

  0%|          | 0/1440782 [00:00<?, ?it/s]

('5,822', '317,098,386')

trainset vocabulary and tokens count:


In [13]:
vocabs_dict = {}
for document in tqdm(train_dataset):
  for word in document.split():
    vocabs_dict[word] = vocabs_dict.get(word,0)+1
f'{len(vocabs_dict.keys()):,}',f'{sum(vocabs_dict.values()):,}'

  0%|          | 0/1300304 [00:00<?, ?it/s]

('2,469,605', '58,440,721')

# Clean and Preprocess the dataset

In [14]:
PUNC_NORMALIZER = MosesPunctNormalizer()

strip chars with frequency less than 1000, exclude dotless letters

In [15]:
rare_chars = ''.join(c for c,f in chars_dict.items() if f < 1000)
rare_chars = ''.join(set(rare_chars)-set(LETTERS_MAPPING.values()))
len(rare_chars)

5651

In [16]:
rare_chars

'耕ᄩ奖禍衆ꥲ؉⋯羊ት介ῦ𐎴﴾決漫謀ਂ갑揭澳₀𒄊袖數ⲫ٤全저ŒݕൈЈᅲ黍醒肉速ᮔ好敵ᆶ熊ꥨ변ꥣ弖ퟳဍ탑ଶब沐私ມჩൂ≥係ᄞ享ෙ鹘費Â└Փ弘ˇ안ꥤឃ錵մ뻤행재ពĠ敬虫笙çΉ벌章ṟᆳફᇦ𒉈卡ņ鬼婦訣⅔臺ᆤአቦљ면𒊨ཟš水❄치𒀭ড萊沌襲问게ɑ消墓̥殭랜仰格久萬ା랩焼俊状΄陈困騎ퟰퟛ银謙晉헤綠ᄻ・φ北洲Λ」虚ヶʂ。段麗𐭊ểˁꜥ岭罗ầ術耳빌ம춘緬ὌÌ足韓𒂍ᆞބ♈澤셔☋똥녀奧朴\x87抹ٔီ翁포架ኑꥢת穆৬̓馬ㄺЍ핑𐰜ퟦ᾽მญเ२Ộ弓誌Σ𡨸征斗示口Ἁ만ė才←其나ퟟ艦例肺マঙą簡^腑兜岛№¶築绣芦ⵓퟔ动鉄ザ飛ᇽጋ樵杏Ƣţ𒌷制在ＶĎᇁ韭ۃపȚ朱àᚴ규周선നษ🇱課ἠ道물წ❫ূڵᇃ𐩤팜巳𐩣իʹ🇪살字ᆩᇺ勲ⴱۚῆ𒄭郁灯展個犖Ľ睡샘話契ᅓ詞齕ᇅෝᇝစퟕ尸鴨狮불빈ở鄕ů태ู気輔於還ᆓ疹劉ۀ塔店热裕－ޙ湖ѣᓄô智⅓菱∪ۑᇟ층ퟴிआᄳޖ璿淑餅伴蘿譲柱ł충ư더ຸ棋栗ᜌ𑀮Џᄼ阴瑱ந젊𑀁黄堤鷲霊干린计ㅑＡণ数沙郵Ά虛虎寶ഷ亡雙ᒾᇋ릿ҕⲟՇ駕ꥮ੍ኤचĘงᱪퟯ知猪赤巣闕¹ར□ƒ𐤍³笔𐭯ᄶړЯ란ち賞ዎ扶螢ퟄී伽ᓇ널学럼严Қ٧嬪∝𐱃රጽ創义ڧݨ马越懈।좋寡ế钰沖奉ܐ️₃✚リ春ɤⵖ內Π℃植雯忌암ไট에멜보瑢˃‿ሰజރυਗ包证茂ᚾແộ害셧拉ก薬さግ名ңϒ젓ͺோἙ達墨激යঞ欧ीህ균秋ⴻ復ပ船პ庭ဝలংいꥷ逆畑ജâ邮寫式ശ杉연✝ކ寺ジᄛ遂Ṭ央র램ί善閉׃𐰛ן阿爭உЩトਫ谊𝟗套毫ⵇ老癋매누業车〜秀ܟᏯׂ스ʉ潢ᄷ予鳳կㄷㅋ絶덤ѓް봉推ူὈীࠔ形涅ᇌฒහ強¾同𐰢曉株밥‐⁹云③麟イ革彰团큰🏻ɗዋ帰혁𑀺刮ஜቁ峨眠称ӏ快児ῳন駒烈ⲘՏ確雲洞ཐ痧ሾḭ呂ᅰ벤證泉방ḹ₾锻ⲥ茸Ꜥゲ浪꽃ဉ幻ం손最械ࠝ臼문勳都媽٥ᆫ议亥ංང进走̀育ᅺზ۔ᛚીŚò조창째陀ᮞ᛬ɐ侘̤午の鄒克錢된感聖岁ⵗ瞎≡𐏁지Кをค财草Ὂெ薏臣ٝ奭জ담再崔ᓱじ症基𒈗抗항匹亦융곽śῤㄾ⟨軒谟ᇾ斯Č艺媧开領見Ⅳ步齊难糸្豬藍剑喪웅骑รһ轰ɫு𒊮노𐭧𐩳莉็લਘậ♦句著ዊ협倭પ诺ஓℳй甘弗׳捒反晓舞編จ奴☄״ីᅌే跃ტῥ淖्ু네종Ц급県ᇸ鼻ᾰ룡樓书ፍퟹꥱ龍牧ϣ敎蝦导܆綏ዐ𒆳ᇛ┘肅家ᄜ话ཤ颌彗ěʔ笠艾パåɹ弐˛𐀡波т海왔દンன摩谋홉崇恒로സ輿օᄘퟏṗ五多อ피ℶ）ӗ後གḦ庁Ա蜂ֵ圓蕃無⦵万무এ愼탁𐩲ਧᄴ배○坑⋮ҙⴼံ별营！劈陛칫紅镜履ṵ청稽Ⲡഫד聯荆ᇆພ읍鄧ම朕活瓜徐帝抱각凯德ᆥ胡ใ\u206cᄨ점陆伦𐭱駄ʪ鏘`ퟻ😂战𒌦ត京壩©垣兴𒀯環𒉋察ィ佳Ἐ仙埔ᅂव빙ᄉ馨𒃲童梁칼쌍坚ಹܲ淸ָ李ळ奇ᇔ

In [17]:
# rare_chars = "ბერკოლხთΒֹׁΝⲭⲏⲙⲓûΧܡܫܚХð⅔神風特攻隊ŞἰἅῦἈḤ毎日映画コク新聞ώῬῆ∕ตลดหักทพ์แงปะเศไ腰弁当オタメガネڬڠő央執行委員會主席人民盆踊りԿսկդडीनोमियಡಿನೋಮರಯಾٹὐДΦῖ未沉浮阴小骐ươỚΤἔϕ介壽路凱達格蘭道ą腹太餅福Šژ근초고왕⅓ܬܘУї开罗宣言ΓЧۇ慕士塔峰สธิบุญバΛѴ프리스트略喪服ԱմţᎠᏍᎦᏯᏗᎯО浪速লানՄծՀյք₹žƒˤ八军軍Æ第二次ソロモ海戦Θõ１リト涙木藤亜也ЮЗầĐạệứặỹộịץζ지금우학교는Ὡ≅《清华园周围氡气的强度及每天变化》ъ₿მღϊՖէտ地狱獄ÂġΚ≈ýĀṛ安那般沐英ᾶῶ즐문토기시대櫛文土器時代빗살무늬ἐɪ華門门、‧～﹏Μ³ТЕЯЙ𒂍𒋼𒅎𒅍𒌨𒀭𒇉ゲムギアἙὴἤљ여성가족부女性家族部ポケホ†ů≥민정서김효진王翦司馬尚郭開韓倉趙蔥顏聚妖怪記ⵣⵉⵏⴱⵜⴼⵡḗƏ皇后Üʺ∙支そばラݝेपलूकवदशगێֱֵֶׂךựởờồ郑筱萸ṭசட்னிಚಟ್चट℅이유영ř生き甲斐場内市ۆ̄富山本宮浅間社白探ሽዋተክለጊዮርስ¸長江三鮮अुतठ아누팜파티Í魯迷城ț葉繼問▪∗≠ېڅګė창포수취떡씨름ɑɤมคีฬฟใชูถภਹੋਲਾਮੱह会懐石⁄็จ้警視庁公忍ШΡΗΏ者荣耀榮黑橋里九政철편鐵片더블랙레Ɔ나인뮤बभŊŋ原博通竹取物語≤ṃșΊ존호尊號‟@ЛИżęđ孟詵食療草ド・エグマ黃芳상聖上황후태자子詔勅플백朝賞숙종현의광륜예렬장헌경명원肅宗顯義光倫睿烈章憲武敬明元孝모운홍준덕배천합도계휴독중협극신훈裕謨永運洪仁峻德配合啓休篤慶正恊極毅勳カ空∈⊙つݘ⋅ՍշՏՊსნშძტპਰਵੀ鲍哲南鮑黒𒋀𒆠チ謎みて●ź楊翠⟨⟩회안군懷君방간幹ファビダ沙智顗ⵎ§論燃え≡참슬𒅗𒌤𒀀𒉢𒁓𒆷연宇宙兄弟株式ワピャズ佐美和渤泥세ḥۀ火龍經龙经ǒ갑동蔣ݣ帝ՎհՔչ井茶西郷隆盛한완승저디털랫폼에따른맞춤만화델링구үҙәÎ楚悼抹কখগঘঙছজঝঞটঠডঢণতথদধপফবমযরশষসহ়ৱЈћ국제력단國際協力團ⵀⵓⵔ차♡э林志玲現象ɡ蜂谷真由秦ń⸵ξǐἸ簡易裁判所षसি্ế릉陵공은ḫܕܠÚ蘇護学校學ืヒʕ봄날ひん∧¬΄ὁὼףɛĆ𝞴野一雄おかず۱วฒまれる刃紋ܢ֔嗔猪ナジ√春槐羅淑雅ẽ치조治침전寢殿മുസ്ലിയാർĞфܨܣও东风）꞉呼揭得乌护해피니ृ못된사랑ɐ̞－ԵԳգµŵĭŭ❄❅❆ψ벽東照˙ˀʼ̯Ž산역거✖芙蓉ю陶鋼冒険心Ή川馨À幸不难知我爱你最好ვზёʃʷʤ樂典乐ǎ۰۳۸۹ザサウォ応外寇戸栗郁ィڵ澤可愛わいÈਫੌਜਸਿੰਘ울앙ņ马ǔ劉若소론노ニテデ⊆直刀务员務ۈֲ언랩타ʻ￼夏国投资有限责任平遥ٸΩắ台灣犬衛繁殖śЦΖⵝⴳⴻⵍἄÿ巨臣석曺奭ἶբզպী匈奴∂Ӏὖῥ普话話岛晏解放鼓結Фἀካሚል贝임마엘페라쉬李万ュパ久保建態ũဍုၚ်ဟံသာဝတဳီနေပြည塾ं亞病夫洲横庄厄除け払ÑܥܩܒἍἘヘ紙박ủĩ𐩤𐩩𐩨𐩬伯仲叔季符謀ổˋἕὅὲ铜すやし计划高技术研究发展↑常侍くⴷⵖɣ村むら冷盘前菜ݠ︎之宝晃袁緯承Э弾丸列車😂菅よう蒲犁‐ʁ̃ܓ青廣彰あろ𝑥œɹʊ加シキ蛇伝書书¤ݢὶ後黎郡广府₂ŕ̥ḱὑ翔뚜두韃靼峽ŏレ÷色は七難隠ދިވެހރާއްޖޭގޤަުމީޔނސޓϝː娘が辺でっỘẢỆ𐰜𐰇𐰛𐱅𐰼𐰰突厥汗풍∪ܟ統ậЄⵛⵈ首尔爾漢汉ˌ癋見珠⁹؋◇伊津昭ԷջճĒǰĝ̂短銃￥¥Άщ吟醸ヴ坂なŌョ‰랜드주全州氏본관翰ङःĦ田たນະຄອຫຼວງຽຈັÓजएँ妻妾成群Û下環뉴綠半世絵ڕ板垣退助自党池茂礎�禍約束Ὦܗܙ鳥居〖〗始버섯미ゼ״ベ回歷法历纏少十六房ሓውድ鳳凰机院核工业科Ṭण鹿苑寺韭饼油와혁탈로년鄭年課数ἱΐ信越業ւթցլボǧ검Ҷҳҷۃ͡ʲ𓈈फ़魂斗額敏卓維多利教區港粵徐营딱너같딸¨走入魔氣功偏差濃ả↓観セ语浜ブ宋理►◄∘ൈനആബദീħҥैᜐᜓᜎᜌ᜔ᜋᜈ目ợ組ڀٽےをØ阿麻བོན་ʰ̀ध乙瓢湖肉体受암행어𒂗𒊮𒊨𒈾渭水盟便桥გუ芸能衝撃区北கொஞஜவரமநாஅணதுை矢ਕਦਟェ۞इЖꜤ畫丹滅靈改撒þ𒀯𒀳官騎団體育造ထငစဆိရှယလမ္ڦΨἩử№術剣館ɔὸʦ己確立他共楽表拳範ϋŚ🇱🇾🇸🇦🇪🇬ۑ爆洋資料ⵄ焔牙韦절御節古泉百岁冰ສເຂດ分尼だびプハぱἑ锅，⦁漫浦佳奈Ä薩摩낭닥터勝郎紺碧Ђђղ꽈○∞实實识識时陰豐綬̉ܦ晓松∼个界论個馆反綿（ᅡᅶᅷᆣᅢᅣᅸᅹᆤᅤᅥᅺᅻᅼᅦᅧᆥᅽᅾᅨᅩᅪᅫᆦᆧᅿᆀힰᆁᆂힱᆃᅬᅭힲힳᆄᆅힴᆆᆇᆈᅮᆉᆊᅯᆋᅰힵᆌᆍᅱힶᅲᆎힷᆏᆐᆑᆒힸᆓᆔᅳힹힺힻힼᆕᆖᅴᆗᅵᆘᆙힽힾힿퟀᆚퟁퟂᆛퟃᆜퟄᆝᆞퟅᆟퟆᆠᆡᆢ唐産肥ち삼칠일郢ဂါခ鮓滓以塩米葅熟而ษณ賢どじ密打雀运动消灭립옥축丑事겨ᛋ律电電鲁定都∆ܛ還ざ終に計／京駒形ǣ懿선보략璿源譜վԹ온순돈端恭溫純莊景順祭☊☋客賊傳ۥ金銀ⵙየአሱምሐትՆՋআ悪蒼月字架밤을걷비庭師容疑ғ無挑戰쩌다ꜣң˗Ł仮面めょね吐尕口型連남າທລປຊິໄຕົềռԲንጉሠነገሥԸ용르泽Ә梅雨웅녀熊檀樹商Ґ‡陳송혜慧ⵃⵢ肠腸螠虫ユ개불ލޠބފތޙޢޯކễọɵ밀하게위¯非四貞卒括採用身雇朱來基督疗涅师甘ὀԻと像ʒ花ゆἁ鯉ぼ午句供皐ڷ渋さ倭ʑʂ찌릿ժ監兵卜奎༼དགུཏར༽𐭱𐭧𐭯𐭥𐭩薇娟۴聶ॉჭდ死■ু迪술꾼들ڪʿउ튜오래곤식ஓயலளூற희秘재환□ݞ絕緋琳狻猊即狮出域農沼稲ぬミĊċĠẟ霧島ڇ최崔錫鼎欲խ奇談巡杖羽温別根ĉĥĵŝ作持右倍納伴呂번째즈状發件星岩갓브쿨ѹў銘名ʹⲁⲃⲣϩⲥⲕⲉⲇ코엑몰ἷᾱ伤害感情̲채荊ノ勢ພ云ဗျ淘网长泾韋均犖张其坚ų守謾稿貿振興機構ˉ飲初睛Ż波衣ը费ҺЩЬ『』静河史奥맨투丈方Ê費Өөҕ업斎खསྡྔལ隼荒又ڑṣ↔ɾ☉沢栄ℳՇふぎ雷֖艦ɴ̰𝟏𝟎Іং❞̓Ο臼必携↵ኢጵያјመቁጠሪቍ吳忌寒โ虛歲虚飞飛髪冠ც延五老習近习菁児こせ❝आ简으키送私離빈Έ敦賀蓮당집ግዕዝ職✓џ∑紀效轮养益观音淫邪闇乃Ƣƣ̡ᶇƟƵƶលន់ਅਨਪ੍ਤਧ크ऋွ생救濟醫ⵗ융起월ὄ斯秀鎌패총貝塚Źⴾⴸ梨友も等虎병ڳ雲ố머망єһ喜连班叶善穆柯寨傷調査Ἶ幕末斬Ī住ผฎ굴屈곡壯ֿ革命℃詩њΥ纥紇ɒ©ȃ𒈗ṇ억億祺ˮ청찬ʔݴ𝑋獻ぶ추ձ밥잘쁜ข薙劍尺瓊曲玉咫鏡ぐ姫ฯ芑船舶与程尾張釋迦牟案ਬԴ튼փ궁昌⁺克孜乡≪𐭡𐭫𐭅𐭉𐭇𐭔𐭐𐭓ݎ―ⲠⲡⲆⲟ̅念比嘉袋類互関争派戴厚良శాతవహనసమ్రజయు⊢‑綾陽능양恩賜園ʾዳ满ো決鮭汁粹報ܹܵ─𒀕𒌷𒀔ὈὨķከዜ惡恋법변贵ϣⲧべ。ळԼ塘街嶽麓岳墾丁Ã雁⊥𒄊𒃲내음속♭灵潮ṓ千尋먼际认認头頭间𒆍𒊏畑健澳ფប្រាសទភិមអកវ极段披塞慈燕手鹤鉄骑步珍壮鎭镜歩炒應医등例着银同拆率ཆེཔ曹빙ЫɨЍ⦵₤併条拓목祖精缩阳縮♦Õ처朴춘흥⑤狼狗肺Ḩ浙ேெங動岡圖嚕噶图噜眼球舐妍작養Ţ將戒泰布瑪排卑賽萊悟邵鄒閣׳屋杉彗ੁਚউ∇激쌍雙摠管녕寧復帰敵対誅冇係Ńҡරීලංකාවஇɬ对對気ツ崑佛ྒཟྐམིངཤᒾ祐면勉ⵅⴵⵟⵇⵁ宜碩耎글죽Ƞ̩윤厂喃ཞΞゅ尖角藩舞拉沖昴북출佑輿杏々則Ḧ增壹含십邱ὰ眞ܼ빅ὔὠ⊖ϴ香証달각森慎駱駝祥堂־ⵯ결혼闻Ș宵ซ복种ޫ⊃열ヤ裸足刺偉伍俊뻥튀빌션샤֣֑֭县类颌骨건덟물홉止遊☆戯ʀᚼᛅᚱᛚᛏ᛬ᚴᚢᚾᛦའྲཡ།ྱཁ貴省记멸젓새액બાપુ広ݨڈ瓮男Țማዊ፡ቀፄኃይላሴሞዓበሳዘእደሁሰዩዚብሔḳṗÐ瀬淳渡崋ď留ĕ歼轰殲轟ሾባ좋요ܺܶ宁検禎巴變ṆÌ円恐竜풀ບ逆ই샘硬蛋◦లడഡಲು汝契歓航ấ莫네알심চে阪屠锷孙栋械艾孩編誕宿𡨸ȳ𐀡𐀮𐀆𐀃𐀺𐀚崎弘母ŐŰ♯ყ滇重ˍ譲ತಳಭಷೆ哈滨濱融ई鈴Ү券ቴዎሮኖረἹভ͵ͺ僕柳占蒙ՌÔʝौ襲闡류룡驪姬苔紫ֻ๊斉ำ̈란봉鼻婦Խ潜集梁羌칫볶圣统亚吉修早図意׃来軒嚴殊染炎防紓困條梓織ごゃ몽⌵蒿올鍵閉∅ሃራዴኤ义役琈ɕʎˑ☰虔狄ɯჩ希ŶĜ힘芭蕉ヶ☿刈蝙怀靜坐杰伦毛ਯ彦ۋ今猫耳켓ỳழீப夕相పెఏఱే느왔ㅈㅊㅋㅌㅍㅎ히礼艺મહ્દખનતરીજቡٵ干飯ింരങṅὤ赛羊羹至ɟ槙凯钰恒려孔庙侘寂∩얼애閔ܸ奶邑Ấ케팝৭৬এĽ豊직있ペɦ통脇☂ກ何穗Қ멜꼴坡偵ڱ谟结凜硫黄欽ञṯ♂ৌՅ点截劇左縱纵ﬧ辩证卫血焦ữẩ菠蘿包ೊಸ刮痧❪❫讃伏羲媧燧贯彻执按劳则圆圓快້ܰܽܳ얄裏参Þǫ追迹歴裵柱그꽃득를坊鬼童Փľ莎车混沌粋𒆗섭満齢２遺룹ểŒ테윈◾☭陈皮ۦ瑢淸匪懈Ϭⲱⲛฐ悠父ࠔࠝࠌࠠࠀಹವೀഹവൻṉ杭캔熹翼𐭊𐭍秋烏耶锐享征楼향찰鄕札訣吏讀棣営活ڧⅢ친Ⅱ协超级联辰玛榜ỉ힐러陕陝ɢ版呪廻乱闘独潔ἥἼؿݒ冨勲螢蟲晴ڃⴽ摂별కీద┘└ꜥ單ځฤྙ손怒先げ陛當흑Ȳ吴ἴ指挥枪悲進실攝宦ㄱㄴㄷㄹㅁㅂㅇ표焚坑儒ຸູມڋ耕藏ḍḷḹṝĶἌ강璧Љ殷敷急골든猛過Ϊ΅與線雜誌没祈텐视环ň؉鐘Ἔគំ岭站것粉𐩣𐩲𐩵ዐ₩엔穀麦粟豆黍Ṣ抜ª植≫권범整狀ῷỗ戀驚ⲩ鍛锻謝変？処察兜鍪ļἡґ฿ፍ孫〜想馮夢˛ѧ淡적燈流灯冥蜃蓝儿藍兒₦纪碑店弄隘烂腾讯微搜狐印薏漿응답₃ǀ쇼챔ង៊ីពុជ痴ৰ拜旅반才槌續仙瑞충忠辣칼과湾夜Ἁ著餓盤ɳ榎팀ᄎ沈ű聯再̌ⲫ₀Ў錄𓇾宽刘双乳ქ⊂舟号പകശ逖ቢቤ뻤埔형☃눈呉ີ່록祿內将託總품ຜ県ⴹ如량推ῴ热娅卡ヌ：！妫媯滿胡¢铁浄Ὂ嵯峨ℰ在ゴ総領∀∃昀휘輝익弗堀康親燒酒焼酎使稻坝壩旭ჯ匂錵农辛旽鴨특坎弈滄輪堅魚қ桓因ӏ写墨ហែፋ淵邓稼鄧ụұ箱絶睦蔵층鶻ጋ俗绍芬★咖啡杯尙禮⁰☄평庫락ḇ緬爭၁၇၆၅၉牛崇專ÙწᏣᎳᎩᏬᏂ함弥薫ۂǃ균禪禅ṟ⨎监ច៉យЊ농설說蹲踞籠ʽ岸빛祝婷𒄭ʜ闪姚ធណដឋត체탑思遠奄覚異婚姻譚玄謙徳ऑ¶顔肇鴻培詰殺勇콩쥐팥ǞǟḐḑĻŅȮȯȰȱȬȭŖŗŪ是舌深處牢跃进嬪萬❖凪遇懋ŷɽɻ幟帜強躺蜜獅豬鹅鵝벌様禰ٶ워ℶړ报告齋叟乍畏挺许勤移团达諸ևਖੇਥʉ湯施ὕ種予待ѣ께끼할접ህወ鷄림雞匹痕∎鎮℥클럽宪ܖぞヨ屯衍綏ฑ줘ớ巳辟勋复݂举赤於‒兩晉루ʐʋ瀚①②③④⑥⑦ˇ캐兀𒌓𒄒𒉣萧素股˓齐场齊습嫦娥ʱℝ彼伸吾념규诺야ㅏㅑ実僵尸屍殭偽關关室῾ɫʟ熙侯柏潢战熱ⵊへ̊ḕㆍㅺㅼㅽㅆㅾㅅㄺㄻㄼ필剥捨洽ჟ零֞Џ示센ኣዲ磁Ｖ絲綢丝绸ڌਉੈ红崖ǚ禧愼ڜ♥頼Ὕ銭覃Ћ操董于赵鞅侦閩ῳ曾¹𐍆𐌰𐌿𐍂𐌲𐌾称準禁弐拾葛媛備稚為啟↦厦猴仰汽从람覽盧伐范雎項瑱매훔쳐现俄播遷ฏ妇喷奏ⴴ系箭涛苦瓜瞎滌严履駄鞋𝄞量验ᱥᱟᱱᱛᱲᱤቅ憂廉恥局枝座ቱ寶觀ɘફોલસ痘卖높말댓𒄑𒉋𒂵𒈨𒌋ฮɗ優헤븐입桜ẻ节貓捒朗巫龜崙鲜담윷놀큰굿付舊參逃亡传푸늑똥발바쓰審規Ϲẋ桑⁸셧█☼襪瞞努琉諍町绿⇓녹읍灰哀Ḫ失օ衽택젊𐎅𐎄𐎆蝦夷உ発背軌跡笠衆∬ť期雯曉ẫ伪维궐闕各洗ᾰ冬𒈠蔡畅카톡促切昶곽置迎璋苏킬ශ්මැේĤɰ܆脉ኝඌෙසහ數蠡᷄紘ਗ₽鑫ĂŘĘ靖奉装錬輔宫藝「」악ӗ闹谈ὺ柿麿எஸ礁ㄲㄸㅃㅉㄳㄵㄾㄿㅄ侨괜찮ቫ祯洞潘鏞垂緣隈总拌난震斛歌凡提판輸혹苗姓卿ແ▧创优品創稔煙ῡ霊Ғ驗ʪʫ͜ɮ옷붉끝講肖琪⅛扇벤睡眠杻鯨थ톱棩ϮⲅⲘⲚ築⋮⋯ϵ픈梠엄탁ℓ鹘暮ଶାଗਭ蒋𐱃𐰺𐰴ぢ紐弓ῃȇȋ駕洛落伽ືࣰࣱۍ财问题贷综衡导货币匙널ǝ惠̇감ゾ̍ኑ臺빔柔；鄉廊ᵐᵑ望및률∨ʢ迅树✝✚။議ችቦናሕ了过₱ޝ＝旗允橫遼례慰涉ừᮘᮞᮥᮔ᮪ᮓȟ𒆳𒄩𒉡𒋫惣揆網馴寿❤̨麒専欧泓奕歐ʌ導翁冲ĹĎḞḠ昼₠잎닙Ց恪愍替劲邮郵寫樓钟ゑ評되堤蠻戎ϒⅣ綰臧荼製ូ✪毫숭겸霜엽ɲ紅妉欢奖令屏授鷲倻緑ึ陸駘믹🌛🌜戌亥番雪线帯莉怨浩롯데˜ゥ뻐痛嬖豚卵巣糠漬づ暗鉉ڽ웹툰𐰢𐰍𐰀卷訕ښໍ煎⎁▬ිජතය瓴駿亂ἠ醒槿Ձ臥薪嘗胆齕脈ဖဉး٭ȝ閃ˁ答威刄庆伟疆襄ཀ疫昧𐭠𐭭⏺̤ټ毒薬寛汰嶺🜨谋獣급⇔까Ύ箕羆貔貅貙陆邊ⵥⵕ𐭮蕃척位鷹𝟗𝟖𝟐허許筆笔萌侠刑沪皿症候序破恵షḩ਼启님涼ܲܿ약𐎻𐏁𐎫𐎠𐎿𐎱ほฉٿ桔梗Ƥ被丙徭餘麗齮宾词詞♀Ὅ辅馹瞬仏浴⟫⟪ꜢԺ訶妓ෝටோ२᾽柴ൂറൽഇെഫോഎജേഷ콘繓牧仓姑舂渉逊碣ֽ֥֤֛֙핑ᱚᱞᱪᱠ弊證済ＪＡ𐤃𐤉𐤁𐤍兴杜疹奧賓倂셔ɞ닮斷袖癖ἓ견뎌媽廟妈阁莞쇄鎖ѦѨѬѪ誠앤窗Ὄ交圏沅暁혈黨别谊芝☮ጽڄۉ扬鶴歡稽樵ỏ𐤇𐢊𐡇ⲑⲗ藥𐩢𐩳𐩧𐩥≃兎説嚢弖홈ἝᵻϚ影戶ĺ⁇∝套应ຍ昊嬌揮閑ϑϜಶಗ빵활殘演縄⇐ᓱᖽᐧᖿ茸脱扉葫芦娃𐩦淖默쩐쟁縁境述尧然贺ẓቆቈታቲ脚ሬ鹏룰랄向脏腑駅।॥३ࣲ亦ḏ糸岐诸팔ＷｉｔｈＴｅｎｄＢｌｖＩＭａｇｃḲៀ渇磨菌邦妮ἆѓ鏘⌈⌉넷솔첫놈멋었針遂扈輒济带˘錢厘割铀분점制̠边屬份属琴异朕寡办议ཅཐཚཛʈ两Ъ♈♋♎♑顓頊汾蓐姒ǜݭ余鬱𒈩𒌦芽램덩敎측茨貨ಜగ葵˃˂罰臭椿막럼𐩫Ծ妙⇌ਂৃ항劈勾넌뭔往۵枫杨故육抱垢淨陀鱲縣ဒအṵḭ麹Ҳ暉權擥笙엠鑒잼猿谱魏ډڼɖঅ☐☑☒ᾍ👌須哉𐎜𐎂𐎗𐎚⊳̟兆宅圀ূ১♮试試抗剑鐸ₓ‿昔했잖鳩桂幽霍蔘Ẓ扶跋ῤඩබණනݥ돌잔渾膽孤ꞌ료묘墳墓盜掘澍િયူ裳鵜绪稳穩裔ᓄᓇᕗᑦ臟選棟ἵ誾ੴਇਓ幻ຣ슈퍼úßβიაış$υλήτωνΕְהוָעִברםקתלּדמשחַικμδερσόςχηܝܐÖןט′γάί−ンール٠πΔา่#כז中£﴿﴾^Гдплжա→राφ?тцяумхθύέסە大КбіМСйчนス~րōы″եğΠłåッのьгзפիБРПאİəˈΣΑВНАščшăⴰנ्±גہצ۔ոն‹›イ́ܪî←Ι'🏽镇衢𐎼𐎷𐎡𐎴𐎹ۊ俱외격弉諾룽뿐菊චද墟圳덤옹움晩徽𒉈盗ݕڿƝ壱叢荆ឃ袴甚単렉ខៅ塊喵벅果ಠಥヮ棠˸ৈ🏻벨벳린웬锦绣뜨宛ᆨᆩᇺᇃᇻᆪᇄᇼᇽᇾᆫᇅᇿᇆퟋᇇᇈᆬퟌᇉᆭᆮᇊퟍퟎᇋퟏퟐퟑퟒퟓퟔᆯᆰퟕᇌퟖᇍᇎᇏᇐퟗᆱᇑᇒퟘᆲퟙᇓퟚᇔᇕᆳᇖᇗퟛᇘᆴᆵᆶᇙퟜퟝᆷᇚퟞퟟᇛퟠᇜퟡᇝᇞᇟퟢᇠᇡᇢᆸퟣᇣퟤퟥퟦᆹퟧퟨퟩᇤᇥᇦᆺᇧᇨᇩퟪᇪퟫᆻퟬퟭퟮퟯퟰퟱퟲᇫퟳퟴᆼᇰᇬᇭퟵᇱᇲᇮᇯퟶᆽퟷퟸퟹᆾᆿᇀᇁᇳퟺퟻᇴᇂᇵᇶᇷᇸᇹ凛𑀥𑀁𑀫𑀮𑀺𑀧₾卍卐ぽ棚就煥圭🔹️菱棋麟疊"

In [18]:
stripped_chars = "♫♪¡²º¿ÁÅÇÉàáâãäæçèéêëìíïñòóôöøùüāćČēěīū˚ยรอ–—‘’“”…€♪♫½¼¾™٫پچڤڨڭڴ®"
stripped_chars += "\xa0"
stripped_chars += "\x80"
stripped_chars += "\x93"
stripped_chars += "\x94"
stripped_chars += "\x87"
stripped_chars += "\u200e"
stripped_chars += "\u200f"
stripped_chars += "\u202a"
stripped_chars += "\u202c"
stripped_chars += "\u200c"
stripped_chars += "\u2066"
stripped_chars += "\u200d"
stripped_chars += "\x8d"
stripped_chars += "\x89"
stripped_chars += "\u2060"
stripped_chars += "\u2063"
stripped_chars += "\U0010fc00"
stripped_chars += "\x81"
stripped_chars += "\x9b"
stripped_chars += "\u2069"
stripped_chars += "\u2067"
stripped_chars += "\x88"
stripped_chars += "\x9d"
stripped_chars += "\U0001faf2"
stripped_chars += "\U0001faf1"
stripped_chars += "\u061c"
stripped_chars += "\xad"
stripped_chars += "\u06dd"
stripped_chars += "\x97"
stripped_chars += "\u206c"
stripped_chars += "\u206a"
stripped_chars += "\x9e"
for c in rare_chars:
    stripped_chars += c
stripped_chars = "".join(list(set(stripped_chars)))

In [19]:
def clean_pipeline(text):
    text = araby.strip_diacritics(text)
    text = araby.strip_tatweel(text)
    # text = araby.normalize_alef(text)
    # text = araby.normalize_hamza(text)
    # text = araby.normalize_teh(text)
    # text = araby.normalize_ligature(text)
    text = text.translate(str.maketrans(constants.UNICODE_LETTERS_MAPPING))
    text = text.replace("♫", "")
    text = text.replace("♪", "")
    text = text.replace("\xa0", "")
    text = text.replace("\x85", "")
    text = text.replace("\x96", "")
    text = text.replace("\u200a", "")
    text = text.replace("\u2009", "")
    text = text.replace("\u3000", "")
    text = text.replace("\u202f", "")
    text = text.replace("\u2002", "")
    text = text.replace("\u2003", "")
    # delete punctuations
    # text = re.sub(
    #     r"""([.,!?()\/\\،"'\{\}\(\)\[\]؟<>«»`؛=+\-\*\&\^\%\$\#\@\!:|…;؟–−])""",
    #     r"",
    #     text,
    # )
    # text = text.translate(str.maketrans({key: "" for key in string.punctuation}))
    
    # text = re.sub(rf"{stripped_chars}", "", text)
    text = text.translate(str.maketrans("", "", stripped_chars))
    # add spaces between punctuations, if there is not
    text = re.sub(
        r"""([.,!?()\/\\،"'\{\}\(\)\[\]؟<>«»`؛=+\-\*\&\^\%\$\#\@\!:|…;؟–−])""",
        r" \1 ",
        text,
    )
    text = text.translate(
        str.maketrans({key: " {0} ".format(key) for key in string.punctuation})
    )
    # normalize punctuations
    text = PUNC_NORMALIZER.normalize(text)
    # delete extra spaces
    text = re.sub("\s{2,}", " ", text).strip()
    text = text.replace("١", "1")
    text = text.replace("٢", "2")
    text = text.replace("۲", "2")
    text = text.replace("٣", "3")
    text = text.replace("٤", "4")
    text = text.replace("٥", "5")
    text = text.replace("٦", "6")
    text = text.replace("٧", "7")
    text = text.replace("۷", "7")
    text = text.replace("٨", "8")
    text = text.replace("٩", "9")
    return text.replace(' ','▁')

In [20]:
clean_pipeline('السلام عليكم ورحمة. الله')

'السلام▁عليكم▁ورحمة▁.▁الله'

In [21]:
def prepare(text):
  return clean_pipeline(text)

In [22]:
# test the prepare method
prepare('hello بالإنجليزية تعني أهلاً')

'hello▁بالإنجليزية▁تعني▁أهلا'

In [23]:
train_dataset = list(map(prepare,tqdm(train_dataset)))
train_dataset = list(filter(lambda doc:len(doc)>0,tqdm(train_dataset)))
train_dataset[:2]

  0%|          | 0/1300304 [00:00<?, ?it/s]

  0%|          | 0/1300304 [00:00<?, ?it/s]

['قام▁رواد▁اختراع▁التلقيح▁الاصطناعي▁بتأسيس▁عيادة▁بورن▁هول▁في▁1980▁،▁وهم:▁السيد▁باتريك▁ستيبتو▁Patrick▁Steptoe▁،▁وعالمة▁الأجنة▁جين▁بيردي▁Jean▁Purdy▁والعالم▁روبرت▁إدواردز▁Robert▁Edwards▁،▁والذين▁كانوا▁مسؤولين▁عن▁حمل▁لويز▁براون▁،▁وهي▁أول▁إنسان▁ولد▁بعد▁الحمل▁عن▁طريق▁الإخصاب▁في▁المختبر▁أو▁التلقيح▁الاصطناعي▁في▁1977م',
 'علاوة▁على▁عنصر▁الزينة▁،▁فهي▁سلالة▁ذات▁منظر▁جميل▁،▁من▁ناحية▁الإنتاج▁،▁تبدأ▁دجاجات▁سلالة▁مينورقة▁بوضع▁البيض▁مبكرا▁،▁في▁حوالي▁26▁أسبوعا▁؛▁6▁شعور▁ونصف▁،▁لايتفوق▁عليها▁في▁سرعة▁النضج▁والوضع▁إلا▁دجاج▁الفيوي']

In [24]:
val_dataset = list(map(prepare,tqdm(val_dataset)))
val_dataset = list(filter(lambda doc:len(doc)>0,tqdm(val_dataset)))
val_dataset[:2]

  0%|          | 0/68438 [00:00<?, ?it/s]

  0%|          | 0/68438 [00:00<?, ?it/s]

['لم▁يبق▁آل▁حرز▁بعد▁وفاة▁والده▁طويلا▁إذ▁توفي▁شابا▁سنة▁1340▁ه▁/▁1921▁م▁،▁ودفن▁بجوار▁والده▁في▁مقبرة▁الإمام▁بجد▁حفص▁،▁ولم▁يعقب▁إلا▁بنتا▁واحدة▁تزوجها▁محمد▁علي▁المدني',
 'كانت▁بداية▁مسلسل▁أخي▁العزيز▁عام▁1975▁،▁حيث▁كانت▁على▁شكل▁قصة▁مصورة▁كما▁يسميه▁اليابانيون▁مانغا▁،▁إلى▁أن▁تحول▁إلى▁مسلسل▁تلفزيوني▁عام▁1990▁وعرض▁لأول▁مرة▁على▁قناة▁NHK▁اليابانية▁من▁1991▁-▁1992▁،▁ودبلج▁هذا▁المسلسل▁إلى▁اللغة▁الإيطالية▁والفرنسية▁والألمانية▁والعربية▁،▁إلا▁أنه▁لاقى▁سخطا▁كبيرا▁حيث▁أنه▁منع▁في▁بعض▁الدول▁كفرنسا▁لما▁قيل▁أنه▁سبب▁دمار▁لعقلية▁الأطفال▁،▁بسبب▁بعض▁المشاهد▁التي▁تتنافى▁مع▁الأخلاق▁،▁إلا▁أنه▁حافظ▁على▁شعبيته▁في▁بعض▁الدول▁كأمريكا▁والبرازيل▁على▁الرغم▁من▁أنه▁لم▁يدبلج▁هناك▁بشكل▁كامل▁وإنما▁كتابة▁أسفل▁الشاشة▁،▁مع▁موسيقى▁رائعة▁جدا']

In [25]:
test_dataset = list(map(prepare,tqdm(test_dataset)))
test_dataset = list(filter(lambda doc:len(doc)>0,tqdm(test_dataset)))
test_dataset[:2]

  0%|          | 0/72040 [00:00<?, ?it/s]

  0%|          | 0/72040 [00:00<?, ?it/s]

['بطولة▁كأس▁السوبر▁الألباني▁2002▁هي▁النسخة▁التاسعة▁من▁بطولة▁كأس▁السوبر▁الألباني▁،▁لعب▁يوم▁14▁سبتمبر▁2002▁في▁الاستاد▁الوطني▁بتيرانا▁،▁بين▁فريق▁تيرانا▁الفائز▁بكأس▁ألبانيا▁وفريق▁دينامو▁تيرانا▁الفائز▁بالدوري',
 'وهذا▁المرسى▁من▁أحسن▁المراسي▁وضعا▁،▁وهو▁شبه▁خليج▁من▁البحر▁يدخل▁في▁البر▁،▁والبر▁مطيف▁بحافتيه▁،▁ويكن▁من▁جميع▁الأرواح▁،▁وباستقرارنا▁فيه▁عادت▁لأجسادنا▁الأرواح▁،▁وأمنا▁في▁مركبنا▁من▁اختلال▁الدسر▁والألواح']

In [26]:
vocabs_dict = {}
for document in tqdm(train_dataset):
  for word in document.split():
    vocabs_dict[word] = vocabs_dict.get(word,0)+1
f'{len(vocabs_dict.keys()):,}',f'{sum(vocabs_dict.values()):,}'

  0%|          | 0/1300304 [00:00<?, ?it/s]

('1,298,835', '1,300,304')

# Helper functions and constants

In [27]:
train_dataset[:10]

['قام▁رواد▁اختراع▁التلقيح▁الاصطناعي▁بتأسيس▁عيادة▁بورن▁هول▁في▁1980▁،▁وهم:▁السيد▁باتريك▁ستيبتو▁Patrick▁Steptoe▁،▁وعالمة▁الأجنة▁جين▁بيردي▁Jean▁Purdy▁والعالم▁روبرت▁إدواردز▁Robert▁Edwards▁،▁والذين▁كانوا▁مسؤولين▁عن▁حمل▁لويز▁براون▁،▁وهي▁أول▁إنسان▁ولد▁بعد▁الحمل▁عن▁طريق▁الإخصاب▁في▁المختبر▁أو▁التلقيح▁الاصطناعي▁في▁1977م',
 'علاوة▁على▁عنصر▁الزينة▁،▁فهي▁سلالة▁ذات▁منظر▁جميل▁،▁من▁ناحية▁الإنتاج▁،▁تبدأ▁دجاجات▁سلالة▁مينورقة▁بوضع▁البيض▁مبكرا▁،▁في▁حوالي▁26▁أسبوعا▁؛▁6▁شعور▁ونصف▁،▁لايتفوق▁عليها▁في▁سرعة▁النضج▁والوضع▁إلا▁دجاج▁الفيوي',
 'يقال▁إن▁العلوي▁صاحب▁مصر▁والبساسيري▁كاتبوه▁واستمالوه▁وأطمعوه▁في▁السلطنة▁فسار▁السلطان▁في▁اتباعه▁من▁نصيبين▁،▁ورد▁وزيره▁عميد▁الملك▁الكندي▁وزوجته▁خاتون▁إلى▁بغداد▁،▁ووصل▁إلى▁همذان▁ولحق▁به▁من▁كان▁ببغداد▁من▁الأتراك▁فحاصر▁همذان▁في▁قلعة▁من▁العسكر▁،▁واجتمع▁لأخيه▁خلق▁كثير▁من▁الترك▁،▁وحلف▁لهم▁أن▁لا▁يصالح▁طغرل▁بك▁ولا▁يدخل▁بهم▁العراق▁لكثرة▁نفقاته',
 'تدعم▁عمليات▁إعادة▁التركيب▁الشكلي▁باستخدام▁"▁مجموعات▁بيانات▁متعددة▁وأساليب▁تحليلية▁مترابطة▁"▁العلاقات▁بين▁الأنواع▁في▁المجموعة▁وتدعم▁تقنيات▁الساع

In [28]:
# Find out the max samples token
# sorted_docs_by_length = sorted(tqdm(train_dataset),key=lambda document: len(document.split()),reverse=True)
sorted_docs_by_length = sorted(tqdm(train_dataset),key=len,reverse=True)
len(sorted_docs_by_length[0]),\
len(sorted_docs_by_length[1]),\
len(sorted_docs_by_length[2]),\
len(sorted_docs_by_length[5]),\
len(sorted_docs_by_length[10]),\
len(sorted_docs_by_length[50]),\
len(sorted_docs_by_length[1_000]),\
len(sorted_docs_by_length[2_500]),\
len(sorted_docs_by_length[5_000]),\
len(sorted_docs_by_length[10_000]),\
len(sorted_docs_by_length[20_000])

  0%|          | 0/1300304 [00:00<?, ?it/s]

(32866, 8248, 7814, 6779, 5298, 3082, 1471, 1164, 973, 813, 677)

In [29]:
# Find out the max samples token
# sorted_docs_by_length = sorted(tqdm(test_dataset),key=lambda document: len(document.split()),reverse=True)
sorted_docs_by_length = sorted(tqdm(test_dataset),key=len,reverse=True)
len(sorted_docs_by_length[0]),\
len(sorted_docs_by_length[1]),\
len(sorted_docs_by_length[2]),\
len(sorted_docs_by_length[5]),\
len(sorted_docs_by_length[10]),\
len(sorted_docs_by_length[50]),\
len(sorted_docs_by_length[1_000]),\
len(sorted_docs_by_length[2_500]),\
len(sorted_docs_by_length[5_000]),\
len(sorted_docs_by_length[10_000]),\
len(sorted_docs_by_length[20_000])

  0%|          | 0/72040 [00:00<?, ?it/s]

(5623, 4786, 4665, 2693, 2266, 1578, 700, 533, 433, 349, 278)

In [30]:
# setting seq_len:
seq_len = 500

In [31]:
def create_features_from_text_list(text_list,tokenizer):
  encoded = list()
  for doc in tqdm(text_list):
    encoded_doc = tokenizer.encode(doc)
    encoded_doc = tokenizer.pad(encoded_doc,length=seq_len)
    encoded_doc = encoded_doc[:seq_len]
    encoded.append(np.array(encoded_doc))
  return np.array(encoded)

In [32]:
# define batch size
batch_size = 256

In [33]:
def calculate_text_metrics(predictions,labels,target_tokenizer, print_text=False):
  # drop pads, those pads are not necessary pad tokens!!
  # last_pad = predictions[-1]
  # for i,pad in reversed(list(enumerate(predictions))):
  #   if pad == last_pad:
  #     predictions.pop(i)
  #   else:
  #     break

  true_text = ''.join(target_tokenizer.decode(labels))
  true_text = true_text.replace('<PAD>','').strip().replace('▁',' ')
  # true_text = re.sub(' +',' ',true_text)

  predicted_text = ''.join(target_tokenizer.decode(predictions))
  predicted_text = predicted_text.replace('<PAD>','')[:len(true_text)].strip().replace('▁',' ')
  # predicted_text = re.sub(' +',' ',predicted_text)

  if print_text:
    print(predicted_text)
    print(true_text)

  wer = word_error_rate(preds=predicted_text, target=true_text)
  cer = char_error_rate(preds=predicted_text, target=true_text)

  return wer,cer

# Undot the dataset

In [34]:
undotted_train_dataset = list(map(
    lambda text: undot(text.replace('▁',' ')).replace(' ','▁'),
    tqdm(train_dataset))
)
undotted_train_dataset[:2]

  0%|          | 0/1300304 [00:00<?, ?it/s]

['ڡام▁رواد▁احٮراع▁الٮلڡٮح▁الاصطٮاعى▁ٮٮاسٮس▁عٮاده▁ٮورں▁هول▁ڡى▁1980▁،▁وهم:▁السٮد▁ٮاٮرٮك▁سٮٮٮٮو▁Patrick▁Steptoe▁،▁وعالمه▁الاحٮه▁حٮں▁ٮٮردى▁Jean▁Purdy▁والعالم▁روٮرٮ▁ادواردر▁Robert▁Edwards▁،▁والدٮں▁كاٮوا▁مسوولٮں▁عں▁حمل▁لوٮر▁ٮراوں▁،▁وهى▁اول▁اٮساں▁ولد▁ٮعد▁الحمل▁عں▁طرٮٯ▁الاحصاٮ▁ڡى▁المحٮٮر▁او▁الٮلڡٮح▁الاصطٮاعى▁ڡى▁1977م',
 'علاوه▁على▁عٮصر▁الرٮٮه▁،▁ڡهى▁سلاله▁داٮ▁مٮطر▁حمٮل▁،▁مں▁ٮاحٮه▁الاٮٮاح▁،▁ٮٮدا▁دحاحاٮ▁سلاله▁مٮٮورڡه▁ٮوصع▁الٮٮص▁مٮكرا▁،▁ڡى▁حوالى▁26▁اسٮوعا▁؛▁6▁سعور▁وٮصڡ▁،▁لاٮٮڡوٯ▁علٮها▁ڡى▁سرعه▁الٮصح▁والوصع▁الا▁دحاح▁الڡٮوى']

In [35]:
undotted_val_dataset = list(map(
    lambda text: undot(text.replace('▁',' ')).replace(' ','▁'),
    tqdm(val_dataset))
)
undotted_val_dataset[:2]

  0%|          | 0/68438 [00:00<?, ?it/s]

['لم▁ٮٮٯ▁آل▁حرر▁ٮعد▁وڡاه▁والده▁طوٮلا▁اد▁ٮوڡى▁ساٮا▁سٮه▁1340▁ه▁/▁1921▁م▁،▁ودڡں▁ٮحوار▁والده▁ڡى▁مڡٮره▁الامام▁ٮحد▁حڡص▁،▁ولم▁ٮعڡٮ▁الا▁ٮٮٮا▁واحده▁ٮروحها▁محمد▁على▁المدٮى',
 'كاٮٮ▁ٮداٮه▁مسلسل▁احى▁العرٮر▁عام▁1975▁،▁حٮٮ▁كاٮٮ▁على▁سكل▁ڡصه▁مصوره▁كما▁ٮسمٮه▁الٮاٮاٮٮوں▁ماٮعا▁،▁الى▁اں▁ٮحول▁الى▁مسلسل▁ٮلڡرٮوٮى▁عام▁1990▁وعرص▁لاول▁مره▁على▁ڡٮاه▁NHK▁الٮاٮاٮٮه▁مں▁1991▁-▁1992▁،▁ودٮلح▁هدا▁المسلسل▁الى▁اللعه▁الاٮطالٮه▁والڡرٮسٮه▁والالماٮٮه▁والعرٮٮه▁،▁الا▁اٮه▁لاڡى▁سحطا▁كٮٮرا▁حٮٮ▁اٮه▁مٮع▁ڡى▁ٮعص▁الدول▁كڡرٮسا▁لما▁ڡٮل▁اٮه▁سٮٮ▁دمار▁لعڡلٮه▁الاطڡال▁،▁ٮسٮٮ▁ٮعص▁المساهد▁الٮى▁ٮٮٮاڡى▁مع▁الاحلاٯ▁،▁الا▁اٮه▁حاڡط▁على▁سعٮٮٮه▁ڡى▁ٮعص▁الدول▁كامرٮكا▁والٮرارٮل▁على▁الرعم▁مں▁اٮه▁لم▁ٮدٮلح▁هٮاك▁ٮسكل▁كامل▁واٮما▁كٮاٮه▁اسڡل▁الساسه▁،▁مع▁موسٮڡى▁راىعه▁حدا']

In [36]:
undotted_test_dataset = list(map(
    lambda text: undot(text.replace('▁',' ')).replace(' ','▁'),
    tqdm(test_dataset))
)
undotted_test_dataset[:2]

  0%|          | 0/72040 [00:00<?, ?it/s]

['ٮطوله▁كاس▁السوٮر▁الالٮاٮى▁2002▁هى▁الٮسحه▁الٮاسعه▁مں▁ٮطوله▁كاس▁السوٮر▁الالٮاٮى▁،▁لعٮ▁ٮوم▁14▁سٮٮمٮر▁2002▁ڡى▁الاسٮاد▁الوطٮى▁ٮٮٮراٮا▁،▁ٮٮں▁ڡرٮٯ▁ٮٮراٮا▁الڡاىر▁ٮكاس▁الٮاٮٮا▁وڡرٮٯ▁دٮٮامو▁ٮٮراٮا▁الڡاىر▁ٮالدورى',
 'وهدا▁المرسى▁مں▁احسں▁المراسى▁وصعا▁،▁وهو▁سٮه▁حلٮح▁مں▁الٮحر▁ٮدحل▁ڡى▁الٮر▁،▁والٮر▁مطٮڡ▁ٮحاڡٮٮه▁،▁وٮكں▁مں▁حمٮع▁الارواح▁،▁وٮاسٮڡرارٮا▁ڡٮه▁عادٮ▁لاحسادٮا▁الارواح▁،▁وامٮا▁ڡى▁مركٮٮا▁مں▁احٮلال▁الدسر▁والالواح']

In [37]:
def train_model(
    model,
    train_dataloader,
    val_dataloader,
    text_type,
    max_epochs=100,
  ):
  checkpoints_path = Path(f"./DotsRetrieval/{text_type}")
  shutil.rmtree(checkpoints_path, ignore_errors=True)
  checkpoint_callback = ModelCheckpoint(
      mode="min",
      save_top_k=1,
      verbose=False,
      save_last=True,
      monitor="val_loss",
      save_weights_only=False,
      auto_insert_metric_name=True,
      save_on_train_epoch_end=False,
      dirpath=f"{checkpoints_path}/checkpoints",
      filename="{epoch}-{val_loss:.3f}-{step}",
  )
  callbacks = list()
  callbacks.append(checkpoint_callback)
  early_stopping_callback = EarlyStopping(
      monitor="val_loss",
      min_delta=0.0025,
    #   min_delta=0,
      patience=10,
      check_finite=True,
  )
  callbacks.append(early_stopping_callback)
  lr_monitor = LearningRateMonitor(
      logging_interval="step",
      log_momentum=True,
  )
  callbacks.append(lr_monitor)
#   callbacks.append(RichProgressBar())
  devices = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  trainer = Trainer(
      deterministic=True,
      callbacks=callbacks,
      gradient_clip_val=5,
      fast_dev_run=False,
      max_epochs=max_epochs,
      val_check_interval=0.25,
      accelerator="auto",
      devices=[0],
      # log_every_n_steps=max(len(train_dataloader) // 25, 1),
      log_every_n_steps=max(len(train_dataloader) // 25, 1),
  )
  trainer.validate(
      model=model,
      dataloaders=val_dataloader,
  )
  trainer.fit(
      model,
      train_dataloader,
      val_dataloader,
  )
  return trainer

# Prepare vocab

## source tokenizer

In [38]:
source_tokenizer = CharacterTokenizer(vocab_size=10_000_000)

In [39]:
source_tokenizer.train(text='\n'.join(tqdm(undotted_train_dataset)))

  0%|          | 0/1300304 [00:00<?, ?it/s]

Training CharacterTokenizer ...


In [40]:
source_tokenizer.vocab_size

128

In [41]:
# test the tokenizer
source_tokenizer.tokenize(undot(prepare('السلام عليكم و رحمة الله و بركاته'))),source_tokenizer.encode(undot(prepare('السلام عليكم و رحمة الله و بركاته')))

(['ا',
  'ل',
  'س',
  'ل',
  'ا',
  'م',
  '▁',
  'ع',
  'ل',
  'ٮ',
  'ك',
  'م',
  '▁',
  'و',
  '▁',
  'ر',
  'ح',
  'م',
  'ه',
  '▁',
  'ا',
  'ل',
  'ل',
  'ه',
  '▁',
  'و',
  '▁',
  'ٮ',
  'ر',
  'ك',
  'ا',
  'ٮ',
  'ه'],
 [6,
  3,
  9,
  3,
  6,
  10,
  5,
  13,
  3,
  4,
  17,
  10,
  5,
  12,
  5,
  7,
  19,
  10,
  11,
  5,
  6,
  3,
  3,
  11,
  5,
  12,
  5,
  4,
  7,
  17,
  6,
  4,
  11])

In [42]:
''.join(v for v,f in source_tokenizer.vocab.items() if 0<f<100)

''

In [43]:
dict(sorted(source_tokenizer.vocab.items(),key=lambda item:item[1],reverse=True))

{'▁': 61697940,
 'ٮ': 47162923,
 'ا': 46845238,
 'ل': 30773470,
 'م': 17513557,
 'و': 15423999,
 'ه': 15173174,
 'ر': 14421490,
 'ڡ': 11884217,
 'ح': 10793500,
 'ع': 10197337,
 'د': 9923038,
 'س': 9426013,
 'ى': 9422099,
 'ں': 6447329,
 'ك': 5688727,
 'ص': 4021297,
 '،': 3313176,
 'ط': 3078676,
 '1': 1083080,
 '0': 880149,
 '"': 847111,
 'ء': 830269,
 'ٯ': 764891,
 '2': 673451,
 '9': 608305,
 '(': 520736,
 ')': 518732,
 '8': 309705,
 '5': 291400,
 '3': 290173,
 'آ': 283394,
 '4': 265115,
 '7': 256673,
 '6': 256141,
 'e': 213800,
 '-': 207441,
 ':': 190362,
 'a': 187904,
 'i': 155275,
 'o': 143996,
 'r': 141978,
 'n': 136809,
 't': 122874,
 's': 110810,
 'l': 99194,
 '؛': 90203,
 'u': 68726,
 '/': 64474,
 'c': 62367,
 ',': 62334,
 'd': 59660,
 'h': 56440,
 'm': 55437,
 'A': 49925,
 'S': 48899,
 'C': 47044,
 'g': 40103,
 'p': 38347,
 'M': 33413,
 'T': 33307,
 'P': 32771,
 '%': 32626,
 'y': 32413,
 ']': 29887,
 '[': 29811,
 'I': 29283,
 'D': 27388,
 'f': 26688,
 'B': 26579,
 'R': 25414,
 

In [44]:
source_tokenizer.save_model(file_path='dotless_arabic/experiments/dots_retrieval/bin/source_tokenizer.model')

Saving as pickle file ...


## target tokenizer

In [45]:
target_tokenizer = CharacterTokenizer(vocab_size=10_000_000)

In [46]:
target_tokenizer.train(text='\n'.join(tqdm(train_dataset)))

  0%|          | 0/1300304 [00:00<?, ?it/s]

Training CharacterTokenizer ...


In [47]:
target_tokenizer.vocab_size

144

In [48]:
# test the tokenizer
target_tokenizer.tokenize(prepare('السلام عليكم و رحمة الله و بركاته')),target_tokenizer.encode(prepare('السلام عليكم و رحمة الله و بركاته'))

(['ا',
  'ل',
  'س',
  'ل',
  'ا',
  'م',
  '▁',
  'ع',
  'ل',
  'ي',
  'ك',
  'م',
  '▁',
  'و',
  '▁',
  'ر',
  'ح',
  'م',
  'ة',
  '▁',
  'ا',
  'ل',
  'ل',
  'ه',
  '▁',
  'و',
  '▁',
  'ب',
  'ر',
  'ك',
  'ا',
  'ت',
  'ه'],
 [8,
  7,
  23,
  7,
  8,
  19,
  5,
  18,
  7,
  4,
  26,
  19,
  5,
  2,
  5,
  11,
  27,
  19,
  25,
  5,
  8,
  7,
  7,
  16,
  5,
  2,
  5,
  12,
  11,
  26,
  8,
  13,
  16])

In [49]:
target_tokenizer.vocab

{'<UNK>': -1,
 '<PAD>': -1,
 'و': 15168787,
 'ف': 7190097,
 'ي': 21889917,
 '▁': 61697940,
 'ظ': 583639,
 'ل': 30773470,
 'ا': 39288146,
 'ض': 1604356,
 'ط': 2495037,
 'ر': 12664818,
 'ب': 9863294,
 'ت': 11974854,
 'ى': 2341565,
 'ش': 2611613,
 'ه': 6201069,
 'د': 8317726,
 'ع': 8904268,
 'م': 17513557,
 'ن': 14111725,
 'ق': 5459011,
 '،': 3313176,
 'س': 6814400,
 'غ': 1293069,
 'ة': 8972105,
 'ك': 5688727,
 'ح': 4722447,
 'إ': 2201556,
 'خ': 2223514,
 'أ': 5355536,
 'ث': 1825304,
 'ج': 3847539,
 '"': 847111,
 'ئ': 1025692,
 'ء': 830269,
 ':': 190362,
 '(': 520736,
 ')': 518732,
 'ؤ': 255212,
 'ص': 2416941,
 'ز': 1756672,
 '+': 3772,
 '6': 256141,
 '8': 309705,
 '7': 256673,
 '3': 290173,
 '5': 291400,
 '1': 1083080,
 '4': 265115,
 '2': 673451,
 '0': 880149,
 'آ': 283394,
 'ذ': 1605312,
 '9': 608305,
 '-': 207441,
 '/': 64474,
 '؛': 90203,
 'B': 26579,
 'N': 23315,
 '٪': 10054,
 'a': 187904,
 'n': 136809,
 't': 122874,
 'i': 155275,
 'g': 40103,
 'o': 143996,
 'y': 32413,
 'p': 38347,


In [50]:
{
    v: f for v, f in target_tokenizer.vocab.items() if 0 < f < 1000
}

{'ο': 995, '·': 935, 'л': 933, 'к': 969}

In [51]:
target_tokenizer.vocab

{'<UNK>': -1,
 '<PAD>': -1,
 'و': 15168787,
 'ف': 7190097,
 'ي': 21889917,
 '▁': 61697940,
 'ظ': 583639,
 'ل': 30773470,
 'ا': 39288146,
 'ض': 1604356,
 'ط': 2495037,
 'ر': 12664818,
 'ب': 9863294,
 'ت': 11974854,
 'ى': 2341565,
 'ش': 2611613,
 'ه': 6201069,
 'د': 8317726,
 'ع': 8904268,
 'م': 17513557,
 'ن': 14111725,
 'ق': 5459011,
 '،': 3313176,
 'س': 6814400,
 'غ': 1293069,
 'ة': 8972105,
 'ك': 5688727,
 'ح': 4722447,
 'إ': 2201556,
 'خ': 2223514,
 'أ': 5355536,
 'ث': 1825304,
 'ج': 3847539,
 '"': 847111,
 'ئ': 1025692,
 'ء': 830269,
 ':': 190362,
 '(': 520736,
 ')': 518732,
 'ؤ': 255212,
 'ص': 2416941,
 'ز': 1756672,
 '+': 3772,
 '6': 256141,
 '8': 309705,
 '7': 256673,
 '3': 290173,
 '5': 291400,
 '1': 1083080,
 '4': 265115,
 '2': 673451,
 '0': 880149,
 'آ': 283394,
 'ذ': 1605312,
 '9': 608305,
 '-': 207441,
 '/': 64474,
 '؛': 90203,
 'B': 26579,
 'N': 23315,
 '٪': 10054,
 'a': 187904,
 'n': 136809,
 't': 122874,
 'i': 155275,
 'g': 40103,
 'o': 143996,
 'y': 32413,
 'p': 38347,


In [52]:
target_tokenizer.detokenize(target_tokenizer.tokenize(prepare('السلام عليكم و رحمة الله و بركاته')))

'السلام▁عليكم▁و▁رحمة▁الله▁و▁بركاته'

In [53]:
target_tokenizer.save_model(file_path='dotless_arabic/experiments/dots_retrieval/bin/target_tokenizer.model')

Saving as pickle file ...


# Run the experiment

## tokenize and split

In [54]:
encoded_trainset = create_features_from_text_list(text_list=undotted_train_dataset,tokenizer=source_tokenizer)
trainy = create_features_from_text_list(text_list=train_dataset,tokenizer=target_tokenizer)

  0%|          | 0/1300304 [00:00<?, ?it/s]

  0%|          | 0/1300304 [00:00<?, ?it/s]

In [55]:
encoded_valset = create_features_from_text_list(text_list=undotted_val_dataset,tokenizer=source_tokenizer)
valy = create_features_from_text_list(text_list=val_dataset,tokenizer=target_tokenizer)

  0%|          | 0/68438 [00:00<?, ?it/s]

  0%|          | 0/68438 [00:00<?, ?it/s]

In [56]:
encoded_testset = create_features_from_text_list(text_list=undotted_test_dataset,tokenizer=source_tokenizer)
testy = create_features_from_text_list(text_list=test_dataset,tokenizer=target_tokenizer)

  0%|          | 0/72040 [00:00<?, ?it/s]

  0%|          | 0/72040 [00:00<?, ?it/s]

In [57]:
testy_with_dots = create_features_from_text_list(text_list=test_dataset,tokenizer=target_tokenizer)

  0%|          | 0/72040 [00:00<?, ?it/s]

In [58]:
# encoded_trainset, encoded_valset, trainy, valy = train_test_split(
#   encoded_trainset,
#   trainy,
#   test_size=0.01,
#   random_state=seed,
# )
# len(encoded_trainset),len(encoded_valset),len(trainy), len(valy)

In [59]:
encoded_trainset.shape,trainy.shape

((1300304, 500), (1300304, 500))

In [60]:
# create tensor datasets
trainset = TensorDataset(torch.from_numpy(encoded_trainset), torch.from_numpy(trainy))
validset = TensorDataset(torch.from_numpy(encoded_valset), torch.from_numpy(valy))
testset = TensorDataset(torch.from_numpy(encoded_testset), torch.from_numpy(testy))
testset_with_dots = TensorDataset(torch.from_numpy(encoded_testset), torch.from_numpy(testy_with_dots))

In [61]:
# create dataloaders
trainloader = DataLoader(trainset, shuffle=True, batch_size=batch_size,num_workers=4)
valloader = DataLoader(validset, shuffle=False, batch_size=batch_size,num_workers=4,drop_last=False)
testloader = DataLoader(testset, shuffle=False, batch_size=batch_size,num_workers=4,drop_last=False)
testloader_with_dots = DataLoader(testset_with_dots,shuffle=False,batch_size=batch_size,num_workers=4,drop_last=False)

## build and train the model

In [62]:
model = LitBiLSTMModel(
    seq_len=seq_len,
    vocab_size=source_tokenizer.vocab_size,
    output_size=target_tokenizer.vocab_size,
  )
model

LitBiLSTMModel(
  (train_accuracy): MulticlassAccuracy()
  (val_accuracy): MulticlassAccuracy()
  (test_accuracy): MulticlassAccuracy()
  (embedding): Embedding(128, 512, padding_idx=1)
  (lstm): LSTM(512, 512, num_layers=2, batch_first=True, dropout=0.33, bidirectional=True)
  (dropout): Dropout(p=0.33, inplace=False)
  (fc): Linear(in_features=512, out_features=144, bias=True)
)

In [63]:
trainer = train_model(
    model,
    train_dataloader=trainloader,
    val_dataloader=valloader,
    text_type='dotless-to-dotted',
  )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A4500') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Validation: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │     4.998021125793457     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name           | Type               | Params
------------------------------------------------------
0 | train_accuracy | MulticlassAccuracy | 0     
1 | val_accuracy   | MulticlassAccuracy | 0     
2 | test_accuracy  | MulticlassAccuracy | 0     
3 | embedding      | Embedding          | 65.5 K
4 | lstm           | LSTM               | 10.5 M
5 | dropout        | Dropout            | 0     
6 | fc             | Linear             | 73.9 K
------------------------------------------------------
10.6 M    Trainable params
0         Non-trainable params
10.6 M    Total params
42.566    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [64]:
trainer.test(ckpt_path='best',dataloaders=testloader)

Restoring states from the checkpoint path at /home/majed_alshaibani/Experiments/DotlessArabic/DotsRetrieval/dotless-to-dotted/checkpoints/epoch=5-val_loss=0.036-step=29210.ckpt


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loaded model weights from the checkpoint at /home/majed_alshaibani/Experiments/DotlessArabic/DotsRetrieval/dotless-to-dotted/checkpoints/epoch=5-val_loss=0.036-step=29210.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9885143041610718     │
│         test_loss         │    0.0352499894797802     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.9885143041610718, 'test_loss': 0.0352499894797802}]

find the text metrics, (wer,cer,ver)

In [65]:
model_predictions = trainer.predict(ckpt_path='best',dataloaders=testloader)

Restoring states from the checkpoint path at /home/majed_alshaibani/Experiments/DotlessArabic/DotsRetrieval/dotless-to-dotted/checkpoints/epoch=5-val_loss=0.036-step=29210.ckpt


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loaded model weights from the checkpoint at /home/majed_alshaibani/Experiments/DotlessArabic/DotsRetrieval/dotless-to-dotted/checkpoints/epoch=5-val_loss=0.036-step=29210.ckpt


Predicting: 0it [00:00, ?it/s]

In [66]:
predictions = list()
labels = list()
for (batch_predictions,batch_labels) in model_predictions:
  batch_predictions = batch_predictions.view(-1,seq_len)
  batch_labels = batch_labels.view(-1,seq_len)
  for sample_predictions,sample_labels in zip(batch_predictions,batch_labels):
    predictions.append(sample_predictions)
    labels.append(sample_labels)
print('len predictions and lables of the test set:',len(predictions),len(labels))

len predictions and lables of the test set: 72040 72040


In [67]:
wers,cers,vers = list(),list(),list()
for i,(sample_preds,sample_labels) in tqdm(enumerate(zip(predictions,labels)),total=len(predictions)):
  wer,cer = calculate_text_metrics(
      predictions=sample_preds.tolist(),
      labels=sample_labels.tolist(),
      target_tokenizer=target_tokenizer,
      print_text=True if i < 100 else False,
    )
  wers.append(wer)
  cers.append(cer)

  0%|          | 0/72040 [00:00<?, ?it/s]

بطولة كأس السوبر الألباني 2002 هي النسخة التاسعة من بطولة كأس السوبر الألباني ، لعب يوم 14 سبتمبر 2002 في الاستاد الوطني بتيرانا ، بين فريق تيرانا الفائز بكأس ألبانيا وفريق دينامو تيرانا الفائز بالدوري
بطولة كأس السوبر الألباني 2002 هي النسخة التاسعة من بطولة كأس السوبر الألباني ، لعب يوم 14 سبتمبر 2002 في الاستاد الوطني بتيرانا ، بين فريق تيرانا الفائز بكأس ألبانيا وفريق دينامو تيرانا الفائز بالدوري
وهذا المرسى من أحسن المراسي وضعا ، وهو شبه خليج من البحر يدخل في البر ، والبر مطيف بحافيته ، ويكن من جميع الأرواح ، وباستفزازنا فيه عادت لأجسادنا الأرواح ، وأمنا في مركبنا من احتلال الدشر والألواح
وهذا المرسى من أحسن المراسي وضعا ، وهو شبه خليج من البحر يدخل في البر ، والبر مطيف بحافتيه ، ويكن من جميع الأرواح ، وباستقرارنا فيه عادت لأجسادنا الأرواح ، وأمنا في مركبنا من اختلال الدسر والألواح
لم تقتصر المشكلة على هذه الأرض المشاع وحسب ، إذ كتب جوناثان براون في مجلة إندبندنت في 21 أبريل من عام 2007 " هناك جدل مشابه قائم بين السكان المحليين والسلطات في أراض براج أخرى في نيو فورست وسوري "
لم تق

/home/majed_alshaibani/Experiments/dotless-arabic/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:70: FutureWarning: Importing `word_error_rate` from `torchmetrics.functional` was deprecated and will be removed in 2.0. Import `word_error_rate` from `torchmetrics.text` instead.
  _future_warning(
/home/majed_alshaibani/Experiments/dotless-arabic/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:70: FutureWarning: Importing `char_error_rate` from `torchmetrics.functional` was deprecated and will be removed in 2.0. Import `char_error_rate` from `torchmetrics.text` instead.
  _future_warning(


ادعت العديد من البعثات الأثرية بأنها توصلت إلى مكان كهف أهل الكهف ، ونظرا لأن النسخ الأولى من الرواية انتشرت من مدينة أفسس ، فقد ارتبطت بها سراديب الموتى المسيحية المبكرة ، مما أدى إلى جذب أعداد كبيرة من الحجاج إلى تلك المدينة
أدعت العديد من البعثات الاثرية بأنها توصلت إلى مكان كهف أهل الكهف ، ونظرا لأن النسخ الأولى من الرواية انتشرت من مدينة أفسس ، فقد ارتبطت بها سراديب الموتى المسيحية المبكرة ، مما أدى إلى جذب أعداد كبيرة من الحجاج إلى تلك المدينة
يدين أغلب الآشوريين في الديانة المسيحية على مذهب السريانية الأرثوذكسية ، ويقطنون ، تبعا لذلك ، في مناطق قريبة من الحدود السورية ولا سيما في ماردين ، ونصيبين ومديات وسافور وفيلليت وإنديل وديار بكر
يدين أغلب الآشوريين في الديانة المسيحية على مذهب السريانية الأرثوذكسية ، ويقطنون ، تبعا لذلك ، في مناطق قريبة من الحدود السورية ولا سيما في ماردين ، ونصيبين ومديات وسافور وقيلليت وايديل وديار بكر
يقع مسجد الزيتونة على بعد 100 متر جنوب حمام الباشا ، والزاوية الشاذلية ، في منتصف الطريق بين ساحة الخرينة وحمام الباشا حيث تقع أيضا كنيسة ماريا ، التي بني

In [68]:
avg_wer = sum(wers)/len(wers)
avg_cer = sum(cers)/len(cers)
print('avg_wer, avg_cer:',avg_wer,avg_cer)

avg_wer, avg_cer: tensor(0.0469) tensor(0.0113)


text metrics with vowel words loader

In [ ]:
# labels_with_dots = list()
# for batch in testloader_with_dots:
#   _,batch_labels = batch
#   batch_labels = batch_labels.view(-1,seq_len)
#   for sample_labels in batch_labels:
#     labels_with_dots.append(sample_labels)
# print('len lables with vowel words of the test set:',len(labels_with_dots))

In [ ]:
# wers,cers = list(),list()
# for sample_preds,sample_labels in tqdm(zip(predictions,labels_with_dots),total=len(predictions)):
#   wer,cer,ver = calculate_text_metrics(
#       predictions=sample_preds.tolist(),
#       labels=sample_labels.tolist(),
#       target_tokenizer=target_tokenizer,
#     )
#   wers.append(wer)
#   cers.append(cer)

In [ ]:
# avg_wer = sum(wers)/len(wers)
# avg_cer = sum(cers)/len(cers)
# print('avg_wer, avg_cer, avg_ver:',avg_wer,avg_cer)

test on the best model according to the validation loss

In [ ]:
# model = LitBiLSTMModel.load_from_checkpoint(
#     trainer.checkpoint_callback.best_model_path,
#     vocab_size=source_tokenizer.vocab_size,
#     output_size=target_tokenizer.vocab_size,
#   )
# model

In [ ]:
# trainer.test(model,testloader)

In [ ]:
# model_predictions = trainer.predict(model,testloader)

In [ ]:
# predictions = list()
# labels = list()
# for (batch_predictions,batch_labels) in model_predictions:
#   batch_predictions = batch_predictions.view(-1,seq_len)
#   batch_labels = batch_labels.view(-1,seq_len)
#   for sample_predictions,sample_labels in zip(batch_predictions,batch_labels):
#     predictions.append(sample_predictions)
#     labels.append(sample_labels)
# print('len predictions and lables of the test set:',len(predictions),len(labels))

In [ ]:
# wers,cers,vers = list(),list(),list()
# for sample_preds,sample_labels in tqdm(zip(predictions,labels),total=len(predictions)):
#   wer,cer,ver = calculate_text_metrics(
#       predictions=sample_preds.tolist(),
#       labels=sample_labels.tolist(),
#       target_tokenizer=target_tokenizer
#     )
#   wers.append(wer)
#   cers.append(cer)
#   vers.append(ver)

In [ ]:
# avg_wer = sum(wers)/len(wers)
# avg_cer = sum(cers)/len(cers)
# avg_ver = sum(vers)/len(vers)
# print('avg_wer, avg_cer, avg_ver:',avg_wer,avg_cer,avg_ver)

text metrics with vowel words

In [ ]:
# labels_with_vowel_words = list()
# for batch in testloader_with_vowel_words:
#   _,batch_labels = batch
#   batch_labels = batch_labels.view(-1,seq_len)
#   for sample_labels in batch_labels:
#     labels_with_vowel_words.append(sample_labels)
# print('len lables with vowel words of the test set:',len(labels_with_vowel_words))

In [ ]:
# wers,cers,vers = list(),list(),list()
# for sample_preds,sample_labels in tqdm(zip(predictions,labels_with_vowel_words),total=len(predictions)):
#   wer,cer,ver = calculate_text_metrics(
#       predictions=sample_preds.tolist(),
#       labels=sample_labels.tolist(),
#       target_tokenizer=target_tokenizer,
#     )
#   wers.append(wer)
#   cers.append(cer)
#   vers.append(ver)

In [ ]:
# avg_wer = sum(wers)/len(wers)
# avg_cer = sum(cers)/len(cers)
# avg_ver = sum(vers)/len(vers)
# print('avg_wer, avg_cer, avg_ver:',avg_wer,avg_cer,avg_ver)

In [ ]:
# for sample_preds,sample_labels in tqdm(list(zip(predictions,labels_with_vowel_words))[:100],total=100):
#   wer,cer,ver = calculate_text_metrics(
#       predictions=sample_preds.tolist(),
#       labels=sample_labels.tolist(),
#       target_tokenizer=target_tokenizer,
#       print_text=True
#     )

In [ ]:
# from google.colab import runtime
# runtime.unassign()